MIT License

Copyright (c) 2021  Pablo Daurell Marina,
                    Belén García Puente,
                    Ela Katherine Shepherd Arévalo,
                    Miquel Vera Ramis,
                    Alberto García Doménech,
                    Fernando Bellot Rodríguez,
                    Mateo González de Miguel


Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
import pandas as pd
import numpy as np

In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  COLLAB = True
else:
  COLLAB = False

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install emoji
import emoji

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
     |████████████████████████████████| 61 kB 360 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 170 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=9c6cabae06487da9151e6894faecd4a6c6718c7fd5d71180c6453008331d90b6
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [4]:
import transformers
from transformers import pipeline
pipe = pipeline("text-classification", model="Newtral/xlm-r-finetuned-toxic-political-tweets-es")

Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [24]:
if COLLAB == True:
  df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')
else:
  df = pd.read_csv('../data/dataset_newtral_ucm.csv')

In [29]:
if COLLAB == True:
  df2 = pd.read_csv('/content/drive/Shareddrives/ETICA/tweet_mods_emojis.csv')
else:
  df2 = pd.read_csv('../data/tweet_mods_emojis.csv')

In [20]:
def comprobar_tweet(tweet : str):
  words = tweet.split()
  hayEmoji = False
  listEmojis = str()
  for i in words:
    if emoji.is_emoji(i):
      hayEmoji = True
      listEmojis = listEmojis + " " + i

  return hayEmoji, listEmojis


In [21]:
column_names = ["twitter_id", "original_text", "slug", "party_slug", "emojis", "toxicity", "toxicity_without_emojis", "sev_toxicity", "sev_toxicity_without_emojis"]
df_wordmod = pd.DataFrame(columns = column_names)
df_wordmod

,twitter_id,original_text,slug,party_slug,emojis,toxicity,toxicity_without_emojis,sev_toxicity,sev_toxicity_without_emojis


In [22]:
tweets_with_emojis_list = list()


In [25]:
for idx, row in df.iterrows():
  tweet = row.text
  aux = comprobar_tweet(tweet)
  if aux[0]:
    text_without_emoji = emoji.replace_emoji(row.text, replace="")
    toxic_scores = pipe(text_without_emoji, return_all_scores=True)
    new_row = {'twitter_id':row.twitter_id, 'original_text':row.text, 'slug':row.slug, 'party_slug':row.party_slug, 'emojis': aux[1], 'toxicity':row.toxicity,'toxicity_without_emojis': toxic_scores[0][0]['score'], 'sev_toxicity':row.sev_toxicity, 'sev_toxicity_without_emojis':toxic_scores[0][1]['score']}
    tweets_with_emojis_list.append(new_row)
    df_wordmod = df_wordmod.append(new_row, ignore_index = True) 

In [26]:
df_wordmod = df_wordmod.drop_duplicates()
df_wordmod = df_wordmod.sort_values('toxicity',ascending=False)

In [28]:
if COLLAB == True:
  df_wordmod.to_csv('/content/drive/Shareddrives/ETICA/tweet_mods_emojis.csv', index=False) 
else:
  df_wordmod.to_csv('../data/tweet_mods_emojis.csv', index=False)

In [ ]:
df_wordmod.head(5)

,twitter_id,original_text,slug,party_slug,emojis,toxicity,toxicity_without_emojis,sev_toxicity,sev_toxicity_without_emojis
956,1403119500384522244,"Ekain Perrino tiene 23 años. \n\nHace días, 13...",gerardo-pisarello-prados,catalunya-en-comu,🌈,0.946050,0.530700,0.976543,0.004655
1088,1412152021155778561,🌈 Los que hoy dicen que matar a un chico de 24...,joan-mena-arca,catalunya-en-comu,🌈,0.881831,0.878469,0.941921,0.006227
587,1383460145892446216,"😭 Cada vez que pienso que esta señora, señoro ...",jose-angel-alonso-perez,partido-popular,😭,0.867858,0.996455,0.657210,0.788988
570,1382341705182105610,"“Cerdos endogámicos y pedantes, inútiles, enfe...",pablo-iglesias-turrion,NaN,👇🏻,0.855327,0.993466,0.809204,0.997177
443,1374444794202615809,Lo que han hecho PSOE y Podemos en Mallorca re...,antonio-pablo-gonzalez-terol,partido-popular,🇪🇸,0.846022,0.995966,0.347208,0.991041


In [ ]:
column_names_emojis = ["emoji", "count", "total_toxicity", "mean", "most_toxic", "most_toxic_text"]
df_wordmod_emojis = pd.DataFrame(columns = column_names_emojis)
df_wordmod_emojis

,emoji,count,total_toxicity,mean,most_toxic,most_toxic_text


In [ ]:
dict_emojis = dict()
for idx, row in df2.iterrows():
  aux = row.emojis.split() ## Por si hay varios emojis
  for j in aux:
    if j not in dict_emojis:
      dict_emojis.update({j: {'count': 1, 'total_toxicity': float(row.toxicity), 'most_toxic': float(row.toxicity), 'most_toxic_text': row.original_text}})
    else:
      if float(dict_emojis[j]['most_toxic']) < float(row.toxicity):
        dict_emojis[j] = {'count': dict_emojis[j]['count']+1, 'total_toxicity': dict_emojis[j]['total_toxicity']+ float(row.toxicity),
                                'most_toxic': float(row.toxicity), 'most_toxic_text': row.text}
      else:
        dict_emojis[j] = {'count': dict_emojis[j]['count']+1, 'total_toxicity': dict_emojis[j]['total_toxicity']+ float(row.toxicity),
                                'most_toxic': dict_emojis[j]['most_toxic'], 'most_toxic_text': dict_emojis[j]['most_toxic_text']}
dict_emojis

{'1️⃣': {'count': 4,
  'most_toxic': 0.24993496,
  'most_toxic_text': '1️⃣ Pacta acercar 5 etarras a la semana\n\n2️⃣ Cede prisiones al PNV\n\n3️⃣ Bildu negocia en secreto privilegios y condiciones especiales para los terroristas\n\nLas maniobras de Sánchez para mantenerse en el poder se superan cada día en indecencia e inmoralidad.\nhttps://t.co/2AQdoWeD9J',
  'total_toxicity': 0.627668915},
 '2️⃣': {'count': 4,
  'most_toxic': 0.24993496,
  'most_toxic_text': '1️⃣ Pacta acercar 5 etarras a la semana\n\n2️⃣ Cede prisiones al PNV\n\n3️⃣ Bildu negocia en secreto privilegios y condiciones especiales para los terroristas\n\nLas maniobras de Sánchez para mantenerse en el poder se superan cada día en indecencia e inmoralidad.\nhttps://t.co/2AQdoWeD9J',
  'total_toxicity': 0.627668915},
 '3️⃣': {'count': 4,
  'most_toxic': 0.24993496,
  'most_toxic_text': '1️⃣ Pacta acercar 5 etarras a la semana\n\n2️⃣ Cede prisiones al PNV\n\n3️⃣ Bildu negocia en secreto privilegios y condiciones especiales

In [ ]:
for item in dict_emojis:
  new_row = {'emoji': item, 'count': dict_emojis[item]['count'], 'total_toxicity': dict_emojis[item]['total_toxicity'],
             'mean': float(dict_emojis[item]['total_toxicity'])/float(dict_emojis[item]['count']),
             'most_toxic': dict_emojis[item]['most_toxic'], 'most_toxic_text': dict_emojis[item]['most_toxic_text']}
  df_wordmod_emojis = df_wordmod_emojis.append(new_row, ignore_index=True)

df_wordmod_emojis = df_wordmod_emojis.drop_duplicates()
df_wordmod_emojis = df_wordmod_emojis.sort_values('mean',ascending=False)
df_wordmod_emojis

,emoji,count,total_toxicity,mean,most_toxic,most_toxic_text
293,😭,1,0.867858,0.867858,0.867858,"😭 Cada vez que pienso que esta señora, señoro ..."
298,🤮,1,0.765317,0.765317,0.765317,"🤮 Bruja, borracha, pervertido, estás mal de la..."
299,🤨,1,0.734334,0.734334,0.734334,Que no... que no son lo mismo....todo el día c...
300,🙋,1,0.657581,0.657581,0.657581,Que levante la mano el que NO se haya reunido ...
302,👷,1,0.605709,0.605709,0.605709,"Mañana, 1 de mayo, tú decides:\n\n🦐 Sindicatos..."
...,...,...,...,...,...,...
105,💙,4,-0.633844,-0.158461,0.301539,"Más apoyo a la familias, más medios para los n..."
204,👄,1,-1.000000,-1.000000,-1.000000,👄 El català ha de ser blindat a totes les llei...
211,🇪🇭,1,-1.000000,-1.000000,-1.000000,"Enviamos esta carta a Joe #Biden, como próximo..."
220,👥,1,-1.000000,-1.000000,-1.000000,👄 El català ha de ser blindat a totes les llei...


In [ ]:
if COLAB == True:
  df_wordmod_emojis.to_csv('/content/drive/Shareddrives/ETICA/tweet_mods_emojis2.csv', index=False) 
else:
  df_wordmod_emojis.to_csv('../data/tweet_mods_emojis2.csv', index=False) 

In [36]:
toxic_emojis_list = ["🤥", "🤮", "⛔", "🌈", "❌", "🤡", "🤬", "😡", "🇪🇸", "🌹", "🚫"]
column_names_emojis_replace = ["previous_emoji", "original_text", "toxicity", "emplaced_emoji", "text_replaced", "toxicity_replaced"]
df_wordmod_emojis_replace = pd.DataFrame(columns = column_names_emojis_replace)
df_wordmod_emojis_replace

,previous_emoji,original_text,toxicity,emplaced_emoji,text_replaced,toxicity_replaced


In [38]:
for idx, row in df2.iterrows():
 tox = row.toxicity
 if float(tox) > 0.2 and tox < 0.5: #only take the tweets with an average toxicity
  for i in toxic_emojis_list:
    tweet = row.original_text
    previous_emoji = row.emojis
    replaced_tweet = emoji.replace_emoji(tweet, i)
    toxic_scores = pipe(replaced_tweet, return_all_scores=True)
    new_row = {"previous_emoji":previous_emoji, "original_text":tweet, "toxicity":tox, "emplaced_emoji":i, "text_replaced":replaced_tweet, "toxicity_replaced":toxic_scores[0][0]['score']}
    df_wordmod_emojis_replace = df_wordmod_emojis_replace.append(new_row, ignore_index = True)



In [40]:
if COLLAB == True:
  df_wordmod_emojis_replace.to_csv('/content/drive/Shareddrives/ETICA/tweet_mods_emojis_replace.csv', index=False) 
else:
  df_wordmod_emojis_replace.to_csv('../data/tweet_mods_emojis_replace.csv', index=False) 